In [1]:
import sys
sys.path.insert(0, '../' )


In [2]:
import m_fri_ldt as fri

###########  test   #############

proof = fri.ldt_gen_proof( [i+5 for i in range(16)] , b'123' , RS_rho=32 )


do a redundent commit for v_f0 here for checking correctness
rho: 32, offset: 0x8000000000000000
|original poly| : 16

### commit phase ###
Assume the evaluate on [0x8000000000000000,|16|x32) has been committed.
|v0|: 512
iteration 0: update : [16] -> [8]:
derive new challenge xi <- H(h_state||3||1) : 0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b
deriving new polynomial of length [8]
ibtfy_1( |v0|:512 , 0x8000000000000000) ->
xi:0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b * |vi|:256 ->
evaluated values generated by 1 stage of ibtfy  [256]
commit evaluated valuse. --> commits[0] <- |mesg|: 128
|commits| = 1
update h_state <- H( h_state|| xi || commit ): b'\x88_B\x9aO1\xe0\xea]{\xe3iGLY~\x89z\x16\xd3\xae\tN\xe9&}\x1e\r\x0b\x14\xfa\xd4'
iteration 1: update : [8] -> [4]:
derive new challenge xi <- H(h_state||4||1) : 0xdc2eac1c69a85252b3d87f0a184a6a8c4d6794faa693c985
deriving new polynomial of length [4]
ibtfy_1( |v1|:256 , 0x4000000000000000) ->
xi:0xdc2eac1c69a85252b3d87f0a184

In [3]:


#def ldt_verify( proof , _poly_len , h_state , Nq = 26 , RS_rho = 8 , verbose = 1 ):

############# test  ##############

fri.ldt_verify( proof , 16 , b'123' , RS_rho = 32 )

######## recovery hash state and challenges ########
iteration 0: [16] -> [8]:
derive new challenge xi <- H(h_state||3||1) : 0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b
new polynomial length [8]
mt.root = commits[0] =  b'R\xaf3O\xf6\xd0\xc1m\x84:\xde\xafpu\x13r)d\x7f7L\xab\xa4\x0e\x8f\xc8;\xfeg#IQ'
update h_state <- H( h_state|| xi || commit ): b'\x88_B\x9aO1\xe0\xea]{\xe3iGLY~\x89z\x16\xd3\xae\tN\xe9&}\x1e\r\x0b\x14\xfa\xd4'
iteration 1: [8] -> [4]:
derive new challenge xi <- H(h_state||4||1) : 0xdc2eac1c69a85252b3d87f0a184a6a8c4d6794faa693c985
new polynomial length [4]
mt.root = commits[1] =  b"\xe6\xf3 \xb2vn*\xf8\xce\x95\xcb\xf1m\x17}:\x01\xd1\xae\x95\x00\x0f<\xf0\xc9'\xfa\xc9\x16\xac\x1d&"
update h_state <- H( h_state|| xi || commit ): b'\xdc\xad\x953\xe0@\x14\xb4l=\xb6\xb6\xfb\x80\x037\xc0\xa7X\x94]\xc3tw\xd1\xf0\xb3\x89\x17\x95kF'
iteration 2: [4] -> [2]:
derive new challenge xi <- H(h_state||5||1) : 0xb65ecc31e0240b0d978dc520e799d3fe79db3382892c2e1
new polynomial length 

True